In [46]:
import matplotlib
import cv2
import re
%matplotlib inline
from IPython.display import Latex

In [67]:
def has_flag(flags, idx):
    return flags & (0x1 << (idx))

def generate_eps_combinations(var_name, expr):
    split = expr.split(sep=var_name)
    flags = 0
    split_locations = []
    combinations = {expr}
    
    if expr == var_name:
        return 'ε'
    
    for i, v in enumerate(split):
        if not v and i < len(split)-1:
            split_locations.append(i+1)
        elif i < len(split)-1:
            split_locations.append(i+1)
    
    while flags < 2**len(split_locations)-1:
        offset = 0
        for i, v in enumerate(split_locations):
            if has_flag(flags, i):
                idx = v + offset
                offset = offset + 1
                split.insert(idx, var_name)
        
        combinations.add(''.join(split))
        split = [v for v in split if v != var_name]
        flags = flags + 1
    
    combinations.remove(expr)
    return list(combinations)

match_lowercase = re.compile('''([a-z])''')
match_variable = re.compile('''(\w_\\{\d+\\}|[A-Za-z])''')

def generate_new_rules(rules, expr):
    variables = match_variable.findall(expr)
    res = False
    
    if len(variables) >= 3:
        rules.add(''.join(variables[len(variables) - 2:]))
        res = True
    elif len(variables) == 2 and match_lowercase.search(expr):
        rules.add(match_lowercase.findall(expr)[0])
        res = True
    
    return res

class ContextFreeGrammar():
    PATTERN = '''(\w_\\{\d+\\}|[A-Za-z]+)'''
    EPSILON = 'ε'
    ALT_EPSILON = 'e'
    START_VARIABLE = 'S'
    
    def __init__(self, start_var=START_VARIABLE):
        self._rules = {}
        self._variables = []
        self._expr = ContextFreeGrammar.PATTERN
        self._match_lowercase = re.compile('''([a-z])''')
        self._re_expr = re.compile(self._expr)
        self._start_var = start_var
        self._u_idx = 0
    
    @property
    def rules(self):
        return self._rules
    
    @property
    def variables(self):
        return self._variables
    
    @property
    def start_variable(self):
        return self._start_var
    
    def add_rule(self, rule, use_alt_eps=True):
        for i, match in enumerate(self._re_expr.finditer(rule)):
            v = match.group()
            
            if not i:
                self.variables.append(v)
                self.rules[v] = []
            else:
                if v == ContextFreeGrammar.ALT_EPSILON and use_alt_eps:
                    v = ContextFreeGrammar.EPSILON
                
                self.rules[self.variables[-1]].append(v)
    
    def _get_eps_combinations(self, var, eps_var):
        buffer = []
        for value in self.rules[var]:
            if eps_var in value:
                buffer.extend(generate_eps_combinations(eps_var, value))
        
        return buffer
    
    def normalize(self):
        # make a new start variable
        self.add_rule('S_{0}->%s' % (self.start_variable))
        eps = ContextFreeGrammar.EPSILON
        
        # remove ε-rules
        found = False
        first_round = True
        
        while found or first_round:
            first_round = False
            found = False
            
            for variable, values in self.rules.items():
                if variable == self.start_variable:
                    continue
                if eps in values:
                    found = True
                    print(variable, 'has epsilon production')
                    values.remove(eps)
                    for k, v in self.rules.items():
                        if k == variable:
                            continue
                        else:
                            v.extend(self._get_eps_combinations(k, variable))
                    break

        # remove unit rules
        for variable, values in self.rules.items():
            found = []
            for value in values:
                if len(value) == 1 and value in self.variables:
                    found.append(value)
            for found_value in found:
                values.remove(found_value)
                if found_value != variable:
                    values.extend(self.rules[found_value])
                    
        # add rules to satisfy A->BC
        new_rules = set()
        found = False
        first_round = True
        terminals = dict()
        
        while found or first_round:
            found = False
            res = False
            
            for variable, values in self.rules.items():
                for value in values:
                    res = generate_new_rules(new_rules, value)
                    if res:
                        found = res
            if found:
                new_rules_map = {key: 'U_{%d}' % (i + self._u_idx) for i, key in enumerate(list(new_rules))}
                for key, value in terminals.items():
                    if key in terminals and key in new_rules_map:
                        new_rules_map[key] = value
                for key, value in new_rules_map.items():
                    if len(key) == 1 and match_lowercase.search(key):
                        if key not in terminals:
                            terminals[key] = value
                
                print(new_rules_map)
                self._u_idx += len(new_rules_map)
                for variable, values in self.rules.items():
                    for i, value in enumerate(values):
                        variables = match_variable.findall(value)
                        
                        if len(variables) >= 3:
                            v = ''.join(variables[len(variables) - 2:])
                            values[i] =  value.replace(v, new_rules_map[v])
                        elif len(variables) == 2 and self._match_lowercase.search(value):
                            for match in self._match_lowercase.finditer(value):
                                values[i] = value.replace(match.group(), new_rules_map[match.group()])
                
                for value, variable in new_rules_map.items():
                    self.add_rule('{}->{}'.format(variable, value))
                
                new_rules.clear()
            
            first_round = False
        
        # remove duplicates
        for variable in self.variables:
            self.rules[variable] = list(set(self.rules[variable]))
            self.rules[variable].sort(key=lambda v: (len(v), v), reverse=True)
    
        
    def __str__(self):
        return str(self.rules)
    
    def render_as_tex(self):
        buffer = []
        for variable in sorted(self.rules.keys()):
            values = self.rules[variable]
            buffer.append('$${} \\rightarrow {}$$\n'.format(variable, '|'.join(values)))
            buffer[-1].replace(ContextFreeGrammar.EPSILON, '\\epsilon')
        return ''.join(buffer)
    
grammar = ContextFreeGrammar()
grammar.add_rule('S->ASA|aB')
grammar.add_rule('A->B|S')
grammar.add_rule('B->b|e')
grammar.normalize()
Latex(grammar.render_as_tex())

B has epsilon production
A has epsilon production
{'SA': 'U_{0}', 'a': 'U_{1}'}


<IPython.core.display.Latex object>

In [68]:
grammar = ContextFreeGrammar()
grammar.add_rule('S->aXT|YbT|UbZ|UWc|PQT')
grammar.add_rule('P->aT')
grammar.add_rule('Q->QT|aQ')
grammar.add_rule('T->cT|e')
grammar.add_rule('U->aU|e')
grammar.add_rule('X->aX|R|e')
grammar.add_rule('R->aRb|e')
grammar.add_rule('Y->Yb|R|e')
grammar.add_rule('V->bVc|e')
grammar.add_rule('W->Wc|V|e')
grammar.add_rule('Z->bZ|V|e')
grammar.normalize()
Latex(grammar.render_as_tex())

T has epsilon production
U has epsilon production
X has epsilon production
R has epsilon production
X has epsilon production
Y has epsilon production
Y has epsilon production
V has epsilon production
W has epsilon production
W has epsilon production
Z has epsilon production
Z has epsilon production
{'bZ': 'U_{0}', 'a': 'U_{1}', 'Wc': 'U_{2}', 'QT': 'U_{3}', 'c': 'U_{4}', 'bT': 'U_{5}', 'Vc': 'U_{6}', 'b': 'U_{7}', 'XT': 'U_{8}', 'Rb': 'U_{9}'}
{'c': 'U_{4}', 'b': 'U_{7}', 'a': 'U_{1}'}


<IPython.core.display.Latex object>